In [1]:
# importing module
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

In [2]:
mydatabase = client['mongodb']

In [3]:
collection_names = mydatabase.list_collection_names()

In [4]:
collection_names

['film',
 'staff',
 'inventory',
 'rental',
 'address',
 'category',
 'customer',
 'language',
 'actor',
 'film_actor',
 'payment',
 'film_category',
 'city',
 'country',
 'store']

In [167]:
QUERY_A = mydatabase.film.aggregate([
   {
     '$group': {
        '_id': 0,
        'Gesamtanzahl der verfügbaren Filme': { '$sum': 1 }
     }
   },
   {
      '$project': {
         '_id': 0
      }
   }
])


In [168]:
QUERY_A.next()

{'Gesamtanzahl der verfügbaren Filme': 1000}

In [155]:
QUERY_B = mydatabase.inventory.aggregate([
    {"$group": {
        '_id': "$store_id",
        'film_set': {
            '$addToSet': "$film_id"
            }
        }
    },
    {"$project": {
        '_id': 0,
        'Store_ID': "$_id",
        'Verschiedene Filme': {
            '$size': "$film_set"
            }
        }
    }])


In [157]:
QUERY_B.next()

{'Store_ID': 2, 'Verschiedene Filme': 762}

Select actor.actor_id, actor.first_name, Count(*) Anzahl, actor.last_name from film_actor
join actor on film_actor.actor_id = actor.actor_id
group by actor.actor_id
Order by Anzahl DESC 
Limit 10

In [482]:
QUERY_C = mydatabase.film_actor.aggregate([
   {
    '$group':{
        '_id': "$actor_id",
        'count': {
            '$sum':1
        }
    }
   },
   {
    '$sort':{
        'count':-1
    }
   },
   {
    '$limit':10
   },
   {
    '$lookup':{
        'from': "actor",
        'localField': "_id",
        'foreignField': "actor_id",
        'as': 'actor_info',
        'pipeline': [{
            '$project': {
                '_id':0,
                'firstname': "$first_name",
                'lastname': "$last_name"
            }
        }]
    }
   },
   {
    '$unwind': '$actor_info'
    },
    {
    '$project':{
        '_id':0,
        'Vorname':'$actor_info.firstname',
        'Nachname':'$actor_info.lastname',
        'Anzahl Auftritte': '$count'
    }
    }
])

In [483]:
QUERY_C = mydatabase.film_actor.aggregate([
   {
    '$group':{
        '_id': "$actor_id",
        'count': {
            '$sum':1
        }
    }
   },
   {
    '$sort':{
        'count':-1
    }
   },
   {
    '$limit':10
   },
   {
    '$lookup':{
        'from': "actor",
        'localField': "_id",
        'foreignField': "actor_id",
        'as': 'actor_info'
    }
   },
   {
    '$unwind': '$actor_info'
    },
    {
    '$project':{
        '_id':0,
        'Vorname':'$actor_info.first_name',
        'Nachname':'$actor_info.last_name',
        'Anzahl Auftritte': '$count'
    }
    }
])

In [485]:
QUERY_C.next()

{'Vorname': 'Walter', 'Nachname': 'Torn', 'Anzahl Auftritte': 41}

Select sum(payment.amount) total, staff.staff_id, staff.first_name, staff.last_name  from payment
join staff on payment.staff_id = Staff.staff_id
group by staff.staff_id

total	staff_id	first_name	last_name
30252.12	1	Mike	Hillyer
31059.92	2	Jon	Stephens

In [515]:
QUERY_D = mydatabase.payment.aggregate([
    {
        '$group':{
            '_id': "$staff_id",
            'total':{
                '$sum':"$amount"
            }
        }
    },
    {
        '$project':{
            '_id':0,
            'Mitarber ID':"$_id",
            'Erlös': {'$trunc':['$total', 2]}
        }
    }
])

In [516]:
QUERY_D.next()

{'Mitarber ID': 1, 'Erlös': 30252.12}

Select customer_id, Count(*) from rental
group by customer_id
Order By Count(*) DESC
Limit 10;

In [565]:
QUERY_E = mydatabase.rental.aggregate([
    {
        '$group':{
            '_id': "$customer_id",
            'count':{
                '$sum':1
            }
        }
    },
    {
        '$sort':{
             'count' : -1
            } 
    },
    {
        '$limit':10
    },
    {
        '$project':{
            '_id':0,
            'Kunden ID':'$_id',
            'Anzahl Entleihungen':'$count'
        }
    }
])

In [566]:
QUERY_E.next()

{'Kunden ID': 148, 'Anzahl Entleihungen': 46}

In [4]:
collection = mydatabase['film']
cur = collection.aggregate([{'$count': "amount of movies"}])
cur.next()

{'amount of movies': 1000}

In [5]:
cur = collection.aggregate([{'$count': "amount of movies"}])
cur.next()

{'amount of movies': 1000}

In [32]:
collection = mydatabase['inventory']
cur = collection.aggregate([{
  "$group": {
    "_id": "$store_id",
    "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "store_id": "$_id",
    "count": 1,
    "_id": 0
  }
}])


In [34]:
cur.next()

{'count': 2270, 'store_id': 1}